In [ ]:
pip install -q yfinance yahooquery setuptools pandas-datareader plotly

In [ ]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [ ]:
### Setting up libraries
import numpy as np
import pandas as pd
import yfinance as yf
import yahooquery as yq
import plotly.express as px
from datetime import datetime, timedelta
from tqdm.notebook import tqdm
import time
from scipy.stats import spearmanr
import statsmodels
import statsmodels.api as sm
from statsmodels.tools.tools import pinv_extended
from google.colab import  drive
import warnings

warnings.filterwarnings("ignore")

from rpy2.rinterface import RRuntimeWarning
warnings.filterwarnings("ignore", category=RRuntimeWarning)

### BLOCKLIST
blocklist = [
    'PETR4.SA'  ## Muita treta envolvida
    ,'VALE3.SA' ## Brumadinho e Mariana
    ,'AZUL4.SA' ## Setor ruim
    ,'GOLL4.SA' ## Setor ruim
    ,'JBSS3.SA'
    ,'AALR3.SA' ## TOP 1 Piores ESG
    ,'PNVL3.SA' ## TOP 2 Piores ESG
    ,'PNVL4.SA' ## TOP 2 Piores ESG
    ,'TRIS3.SA' ## TOP 3 Piores ESG
    ,'BRAP3.SA' ## TOP 4 Piores ESG
    ,'BRAP4.SA' ## TOP 4 Piores ESG
    ,'LAND3.SA' ## TOP 5 Piores ESG
]


# Adjust Cientific Notation - Importante to get correct coefficients
# pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [ ]:
### Captura todas as ações negociadas do dia de hoje
assets = (
    pd.read_html( 'https://www.dadosdemercado.com.br/acoes')[0]
    .assign(
        Ticker = lambda x:x.Ticker + '.SA'
    )
    ['Ticker']
    .tolist()
)

### Calculo do Volume no Ultimo período estabelecido
start = (datetime.today() - timedelta(days=365*4)).strftime('%Y-%m-%d')
end = datetime.today().strftime('%Y-%m-%d')

assets = yf.download(assets, start = start, end = end)

### data tidying - Calculo da Volumetria por Ativo & Filtros (Volume + Duplicados)
assets = (
    assets
    .loc[:,('Volume', slice(None))]
    .droplevel(level=0, axis=1)
    [lambda x: x.index.dayofweek < 5]
    .sum()
    .reset_index(drop=False)
    .rename(columns={0:'Volume'})
    .assign(
        teste1 = (
            lambda x: (
                x.Ticker
                .str.replace('11.SA', '' )
                .str.replace('6.SA', '' )
                .str.replace('5.SA', '' )
                .str.replace('4.SA', '' )
                .str.replace('3.SA', '')
            )
        )
    )
    [lambda x: x.Volume > x.describe(percentiles=[.5, .9]).T['50%'][0]]
    .sort_values(by = ['Volume'], ascending=False)
    .drop_duplicates(subset=['teste1'], keep='first')
    ['Ticker']
    .tolist()
)

# Tickers of Global Indexes
factors = [
    '^GSPC',      # S&P 500 - IVVB11
    'ACWX',       # MSCI - Top Ações mundo ordenado por Cap SEM USA
    'XEM.TO',     # MSCI - Ações Emergentes ordenado por Cap - BEEM39
    'EMB',        # USD Emerging Markets Bond
    'BRL=X',      # USD vs BRL
    'GD=F',       # GSCI ETF commodities - MATB11
]

# Union B3's Assets + Global Factors
assets = factors + assets

[*********************100%***********************]  402 of 402 completed
ERROR:yfinance:
10 Failed downloads:
ERROR:yfinance:['AZEV11.SA', 'AMAR11.SA', 'PINE11.SA']: YFTzMissingError('possibly delisted; no timezone found')
ERROR:yfinance:['BGIP3.SA', 'BMGB4.SA']: Timeout('Failed to perform, curl: (28) Connection timed out after 10001 milliseconds. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')
ERROR:yfinance:['BDLL4.SA', 'CEAB3.SA', 'PINE4.SA', 'DASA3.SA', 'BRSR5.SA']: Timeout('Failed to perform, curl: (28) Connection timed out after 10002 milliseconds. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')


In [ ]:
# Date Range
start = '2010-01-01'
end = datetime.today().strftime('%Y-%m-%d')

# Downloading data & adjusting it
data = yf.download(assets, start = start, end = end)
data = (
    data
    .loc[:,('Close', slice(None))]
    .droplevel(level=0, axis=1)
    [lambda x: x.index.dayofweek < 5]
    .set_index(data.index.tz_localize(None))
    .assign(
        EMB = lambda x: x['EMB'] * x['BRL=X'],
        GDF = lambda x: x['GD=F'] * x['BRL=X'],
        GSPC = lambda x: x['^GSPC'] * x['BRL=X'],
        ACWX = lambda x: x['ACWX'] * x['BRL=X'],
        XEM_TO = lambda x: x['XEM.TO'] * x['BRL=X']
    )
    .drop(['^GSPC', 'XEM.TO', 'GD=F'], axis=1)
    .fillna(method='ffill')
)

# Creating factor data
link = 'https://nefin.com.br/resources/risk_factors/nefin_factors.csv'

factors_df = (
    pd.read_csv(link, index_col=[0])
    .rename(columns=str.lower)
    [['date', 'rm',	'smb',	'hml',	'wml', 'iml']]
    .assign(
        date = lambda x: pd.to_datetime(x['date']),
        mkt = lambda x: x['rm'],
        size = lambda x: x['smb'],
        value = lambda x: x['hml'],
        momnt = lambda x: x['wml'],
        liq = lambda x: x['iml']
    )
    .set_index('date')
    .loc[start:end]
    .dropna()
)
factors_df = factors_df[factors_df.index.isin(data.index)]

# Join both datasets into one
data = pd.concat([data.pct_change(fill_method=None), factors_df], axis = 1)

# Turn all values from lognormal to normal
raw_data = np.log1p(data)

[*********************100%***********************]  192 of 192 completed


In [ ]:
# Factors
factors = [
    'GSPC',         # S&P 500 - IVVB11
    'ACWX',         # MSCI - Top Ações mundo ordenado por Cap SEM USA
    'XEM_TO',       # MSCI - Ações Emergentes ordenado por Cap - BEEM39
    'GDF',          # GSCI ETF commodities - MATB11
    'BRL=X',        # USD vs BRL
    'mkt',
    'size',
    'value',
    'momnt',
    'liq'
]

# Fix Assets List
assets = [item for item in assets if item not in ['^GSPC', 'XEM.TO', 'GD=F', 'ACWX']]

# Create dataframe to save alpha and betas
data = pd.DataFrame()

# Run a linear regression to get alpha
for i in tqdm([x for x in assets if x not in factors]):

  # Select index
  y = raw_data[i].dropna()

  qtd = len(y)

  # Selecting factors
  vars = raw_data[raw_data.index.isin(y.index)][factors].dropna()

  X_sm = sm.add_constant(vars[factors])

  y = y[y.index.isin(X_sm.index)]

  vol = np.std(y)

  # fit OLS model - L1_wt=0 Ridge / alpha=0.0001
  results = sm.OLS(y, X_sm).fit_regularized(L1_wt=0, alpha=0.00025)

  n_model = sm.OLS(y, X_sm)
  pinv_wexog,_ = pinv_extended(n_model.wexog)
  normalized_cov_params = np.dot(pinv_wexog, np.transpose(pinv_wexog))

  final = sm.regression.linear_model.OLSResults(
      n_model,
      results.params,
      normalized_cov_params
  )

  a = np.where(
      i == 'GSPC', 's_p',
        np.where(
            i == '^RUT', 'rsl_2000',
              np.where(
                  i == 'EWJ' , 'top_jp',
                    np.where(
                        i == 'GDF', 'cmmdt',
                          np.where(
                              i == 'GCF', 'gld',
                                np.where(
                                    i == 'CL=F', 'oil',
                                      np.where(
                                          i == '000001.SS', 'sse_china',
                                              np.where(
                                                  i == 'IXIC', 'nsdq', i
                                                  )
                                              )
                                      )
                                )
                          )
                    )
              )
      )

  # Create the last table woth all coefficients
  dt = pd.DataFrame(
    {
     'ticker': a,
     'qtd_dias': qtd,
     's_p': [results.params[1].round(3)],
     'acwx': [results.params[2].round(3)],
     'emm': [results.params[3].round(3)],
     'cmmdt': [results.params[4].round(3)],
     'usd_real': [results.params[5].round(3)],
     'mkt': [results.params[6].round(3)],
     'size': [results.params[7].round(3)],
     'value': [results.params[8].round(3)],
     'momnt': [results.params[9].round(3)],
     'liq': [results.params[10].round(3)],
     'return': [raw_data[i].sum().round(3)],
     'vol': round(vol, 5),
     'alpha': [(results.params[0]).round(5)],
     'r_score': [final.rsquared.round(3)],
     'last_update': [end]
     }
  )

  data = pd.concat([data, dt], ignore_index=True)


data[data['ticker'].isin(['PETR4.SA', 'VALE3.SA', 'ITUB4.SA', 'SBSP3.SA'])]

  0%|          | 0/187 [00:00<?, ?it/s]

,ticker,qtd_dias,s_p,acwx,emm,cmmdt,usd_real,mkt,size,value,momnt,liq,return,vol,alpha,r_score,last_update
1,PETR4.SA,4027,0.032,0.065,0.059,0.152,-0.089,0.499,-0.099,0.455,-0.062,-0.157,1.183,0.02796,-0.00003,0.563,2025-06-15
5,ITUB4.SA,4027,0.042,0.075,0.077,-0.018,-0.060,0.408,-0.067,0.088,-0.053,-0.098,0.983,0.01922,0.00007,0.459,2025-06-15
8,VALE3.SA,4027,0.061,0.118,0.099,0.079,-0.112,0.331,-0.087,0.188,-0.108,-0.141,0.805,0.02491,0.00001,0.336,2025-06-15
77,SBSP3.SA,4027,0.046,0.053,0.035,-0.004,-0.073,0.367,0.024,0.084,-0.019,0.005,2.737,0.02208,0.00055,0.241,2025-06-15


In [ ]:
### Plot Avaliando - O indicador de comportamental passado prediz o futuro?
var = 'alpha'
df_plot = data[[var, 'return']].dropna()
corr, _ = spearmanr(df_plot[var], df_plot['return'])

fig = (
    px.scatter(
        data, x=var, y='return', hover_name='ticker',
        labels={
            var: 'Exposição ao Fator',
            'return': 'Retorno'
        },
        title=f'Relação Exposição ao Fator (Alpha) & Retorno das Ações do Ibovespa',
        trendline='ols',
        trendline_color_override = 'black',
        template='plotly_white'
    ).update_traces(
        marker_size=12,
        marker=dict(color='green'),
        opacity=0.4
    ).update_layout(
        font=dict(size=14), showlegend=False
    )
)

fig.show()

In [ ]:
#%%R
#install.packages('cutpointr')

In [ ]:
#%%R

#library(cutpointr)
#library(tidyverse)
#library(readxl)

#data <- read_csv('teste1.csv', show_col_types = FALSE)

#data <- data %>%
#  mutate(flag_return = if_else(return > 0.1, 1, 0))

# Determinação do ponto de corte ótimo
#best_cut <- cutpointr(
#  data = data,
#  x = momnt,
#  class = flag_return,
#  pos_class = 1,
#  direction = '>=',
#  method = maximize_metric,
#  metric = accuracy
#)

# Exibir resumo das métricas c/ score otimizado
#best_cut[, c(2, 4, 8)]

In [ ]:
### Remoção da BLOCKLIST
data = data[(~data['ticker'].isin(blocklist))]

### Filtros de Exposição à Fatores
resumo = data.describe(percentiles=[.025, .985]).T
resumo = resumo.iloc[:, 4:7]

resumo.columns = ['lower', 'mid', 'upper']

### Filtro final - Relativizado
final_data = (
    data[
          # Maior, mais retorno
          (data['qtd_dias'] >= resumo['mid'].loc['qtd_dias'] * 0.75)

          # Maior, mais retorno
          & (data['s_p'] >= resumo['lower'].loc['s_p'])

          # Menor, mais retorno
          & (data['usd_real'] >= resumo['lower'].loc['usd_real'])
          & (data['usd_real'] <= resumo['upper'].loc['usd_real'])

          & (data['mkt'] <= resumo['upper'].loc['mkt'])

          & (data['size'] <= resumo['upper'].loc['size'])

          & (data['value'] >= resumo['lower'].loc['value'])

          & (data['momnt'] >= resumo['lower'].loc['momnt'])

          & (data['liq'] >= resumo['lower'].loc['liq'])
          & (data['liq'] <= resumo['upper'].loc['liq'])

          & (data['vol'] <= resumo['upper'].loc['vol'])

          & (data['return'] >= resumo['mid'].loc['return'] * 1.25)

          & (data['alpha'] > 0)
      ]
      .sort_values(by=['alpha'], ascending=False)
      .reset_index(drop=True)
)

final_data

,ticker,qtd_dias,s_p,acwx,emm,cmmdt,usd_real,mkt,size,value,momnt,liq,return,vol,alpha,r_score,last_update
0,ISAE4.SA,4027,0.020,0.030,0.035,0.005,-0.016,0.182,0.022,0.041,0.001,0.014,3.825,0.01835,0.00092,0.090,2025-06-15
1,STBP3.SA,4027,0.037,0.044,0.032,-0.006,-0.048,0.260,0.132,-0.004,0.003,0.119,3.841,0.02864,0.00088,0.088,2025-06-15
2,SAPR4.SA,4027,0.031,0.032,0.023,-0.012,-0.052,0.242,0.057,0.070,0.010,0.047,3.263,0.02216,0.00069,0.109,2025-06-15
3,FRAS3.SA,4027,0.031,0.012,0.012,0.013,-0.021,0.179,0.095,-0.007,-0.004,0.071,2.790,0.03954,0.00067,0.021,2025-06-15
4,RANI3.SA,4027,0.044,0.045,0.035,0.004,-0.033,0.179,0.056,-0.006,-0.018,0.036,2.628,0.03086,0.00060,0.035,2025-06-15
5,RADL3.SA,4027,0.037,0.043,0.057,-0.016,-0.043,0.238,0.017,-0.051,-0.023,0.005,2.193,0.02013,0.00058,0.127,2025-06-15
6,SLCE3.SA,4027,0.029,0.043,0.029,0.049,-0.051,0.183,0.089,0.015,0.018,0.029,2.543,0.02271,0.00056,0.076,2025-06-15
7,TAEE11.SA,4027,0.011,0.015,0.022,0.001,-0.013,0.153,0.019,0.029,-0.005,0.004,2.380,0.01491,0.00056,0.087,2025-06-15
8,SBSP3.SA,4027,0.046,0.053,0.035,-0.004,-0.073,0.367,0.024,0.084,-0.019,0.005,2.737,0.02208,0.00055,0.241,2025-06-15
9,EQTL3.SA,4027,0.038,0.038,0.043,-0.013,-0.033,0.252,0.052,0.014,-0.015,0.011,2.460,0.01892,0.00053,0.154,2025-06-15


In [ ]:
# Select tickers to get fundamentalist informations #'DEXP3.SA'
lista = final_data['ticker']

# Create dataframe to save fundamental indexes
data = pd.DataFrame()

for ticker in tqdm(lista):
  ### Get the Historical Company Performance - Gross and Net Margin
  #################################################################
  yf_data = yq.Ticker(ticker)
  asst_data = yf_data.history(period = '10y').reset_index(0)
  asst_data['year'] = pd.to_datetime(asst_data.index.to_series(), errors='coerce', utc=True).dt.year

  if 'dividends' not in asst_data.columns:
    asst_data['dividends'] = 0

  ### Get the last price of each year
  time.sleep(0.1)
  last_prices = asst_data.groupby('year')['close'].agg(['last'])

  ### Get the Historical Dividend Yield
  #####################################
  sun_div = asst_data[asst_data.dividends != 0].groupby('year')['dividends'].agg(['sum'])

  ### Grouping Last Price with Dividends Sum
  asst_div_data = pd.concat([last_prices, sun_div], axis=1)

  asst_div_data['yield'] = asst_div_data['sum'] / asst_div_data['last']

  hist_div = round(asst_div_data['yield'].median() * 100, 2)


  ### Final Dataset - Fundamentalist Performance & Index
  ######################################################

  final_data_fund = pd.DataFrame(
      {
      'ticker': ticker,
      'Div. Yield Med': [hist_div],
      }
  )

  time.sleep(0.1)

  data = pd.concat([data, final_data_fund], ignore_index=True)

  0%|          | 0/56 [00:00<?, ?it/s]

In [ ]:
final_data = final_data.merge(data, on='ticker', how='left')[(lambda x: x['Div. Yield Med'] >= 2.5)]

final_data.head(59)

,ticker,qtd_dias,s_p,acwx,emm,cmmdt,usd_real,mkt,size,value,momnt,liq,return,vol,alpha,r_score,last_update,Div. Yield Med
0,ISAE4.SA,4027,0.020,0.030,0.035,0.005,-0.016,0.182,0.022,0.041,0.001,0.014,3.825,0.01835,0.00092,0.090,2025-06-15,8.97
2,SAPR4.SA,4027,0.031,0.032,0.023,-0.012,-0.052,0.242,0.057,0.070,0.010,0.047,3.263,0.02216,0.00069,0.109,2025-06-15,5.23
3,FRAS3.SA,4027,0.031,0.012,0.012,0.013,-0.021,0.179,0.095,-0.007,-0.004,0.071,2.790,0.03954,0.00067,0.021,2025-06-15,3.06
4,RANI3.SA,4027,0.044,0.045,0.035,0.004,-0.033,0.179,0.056,-0.006,-0.018,0.036,2.628,0.03086,0.00060,0.035,2025-06-15,5.13
6,SLCE3.SA,4027,0.029,0.043,0.029,0.049,-0.051,0.183,0.089,0.015,0.018,0.029,2.543,0.02271,0.00056,0.076,2025-06-15,4.44
7,TAEE11.SA,4027,0.011,0.015,0.022,0.001,-0.013,0.153,0.019,0.029,-0.005,0.004,2.380,0.01491,0.00056,0.087,2025-06-15,8.27
10,SHUL4.SA,4027,0.027,0.036,0.020,0.019,-0.049,0.229,0.098,0.051,-0.016,0.052,2.381,0.02343,0.00053,0.096,2025-06-15,4.59
11,PSSA3.SA,4027,0.026,0.039,0.049,-0.016,-0.062,0.230,0.015,0.006,-0.030,-0.003,2.448,0.01830,0.00051,0.143,2025-06-15,4.37
13,BBSE3.SA,3162,0.023,0.045,0.064,-0.010,-0.046,0.306,-0.019,0.004,-0.043,-0.052,1.579,0.01792,0.00044,0.263,2025-06-15,5.87
14,VIVT3.SA,4027,0.028,0.031,0.042,-0.004,-0.021,0.198,-0.007,0.046,-0.023,0.012,2.160,0.01758,0.00044,0.120,2025-06-15,8.49


In [ ]:
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

corr = raw_data[final_data.ticker].corr(method='spearman')
mask = np.triu(np.ones_like(corr, dtype=np.bool_))
corr = corr.mask(mask)

fig = ff.create_annotated_heatmap(
    z=corr.to_numpy().round(2),
    x=list(corr.index.values),
    y=list(corr.columns.values),
    xgap=3, ygap=3,
    zmin=-0.5, zmax=1,
    colorscale='earth',
    colorbar_thickness=30,
    colorbar_ticklen=3
)

fig.update_layout(
    width=1200, height=1200,
    xaxis_showgrid=False,
    xaxis={'side': 'bottom'},
    yaxis_showgrid=False,
    yaxis_autorange='reversed',
    template='plotly_white'
)

fig.show()

In [ ]:
# Tickers of Global Indexes
assets = final_data['ticker']

# Load all data
data = pd.DataFrame()

for i in assets:
  df = (
      yq.Ticker(i)
      .history(start = start, end = end, interval = '1d')
      .reset_index(0)
      [lambda x: pd.to_datetime(x.index).dayofweek < 5]
      [['adjclose', 'dividends']]
      .fillna(method='ffill')
  )

  data[i] = df['adjclose']
  data['div_' + i] = df['dividends']


data.index = pd.to_datetime(data.index).tz_localize('UTC')
data = data.fillna(method='ffill')


### Reinvestimentos
amount_asset = 100000000 / len(assets)

for c in data[assets]:

  data['qtd_pst_' + c] = round(amount_asset / data[c].head(1), 0)

  for i in range(1, len(data)):

    if data['div_' + c][i-90] > 0:
      data['qtd_pst_' + c][i] = round(
          data['qtd_pst_' + c][i-1]
          + (data['div_' + c][i-90] * data['qtd_pst_' + c][i-1] * 0.875 / data[c][i])
          , 0
        )
    else:
      data['qtd_pst_' + c][i] = data['qtd_pst_' + c][i-1]

  data['value_' + c] = data['qtd_pst_' + c] * data[c]


div_data = data[data.filter(like='value').columns]
div_data.index = pd.to_datetime(div_data.index)
div_data.index = div_data.index.tz_convert('UTC')

div_data.pct_change().sum().sort_values(ascending=False)

,0
value_ISAE4.SA,25.445584
value_SYNE3.SA,8.326006
value_FRAS3.SA,6.659015
value_VIVT3.SA,6.542605
value_RANI3.SA,5.930041
value_CMIG4.SA,5.654952
value_TAEE11.SA,5.621557
value_SAPR4.SA,5.441761
value_SLCE3.SA,4.715925
value_PCAR3.SA,4.483383


In [ ]:
assets = final_data['ticker']

# Load all data
data = pd.DataFrame()

for i in assets:
  df = (
      yq.Ticker(i)
      .history(start = start, end = end, interval = '1d')
      .reset_index(0)
      [lambda x: pd.to_datetime(x.index).dayofweek < 5]
      [['adjclose', 'dividends']]
      .fillna(method='ffill')
  )

  data[i] = df['adjclose']
  data['div_' + i] = df['dividends']


data.index = pd.to_datetime(data.index).tz_localize('UTC')
data = data.fillna(method='ffill')


### Reinvestimentos
amount_asset = 100000000 / len(assets)

for c in data[assets]:

  data['qtd_pst_' + c] = round(amount_asset / data[c].head(1), 0)

  for i in range(1, len(data)):

    if data['div_' + c][i-1] > 0:
      data['qtd_pst_' + c][i] = round(
          data['qtd_pst_' + c][i-1]
          + (data['div_' + c][i-1] * data['qtd_pst_' + c][i-1] * 0 / data[c][i])
          , 0
        )
    else:
      data['qtd_pst_' + c][i] = data['qtd_pst_' + c][i-1]

  data['value_' + c] = data['qtd_pst_' + c] * data[c]


div_data = data[data.filter(like='value').columns]
div_data.index = pd.to_datetime(div_data.index)
div_data.index = div_data.index.tz_convert('UTC')

div_data.pct_change().sum().sort_values(ascending=False)

,0
value_FRAS3.SA,6.103603
value_ISAE4.SA,4.479572
value_RANI3.SA,4.464062
value_SAPR4.SA,4.204787
value_PTBL3.SA,3.614177
value_SLCE3.SA,3.533311
value_PCAR3.SA,3.514482
value_SHUL4.SA,3.426168
value_DIRR3.SA,3.281414
value_POMO4.SA,3.261429


In [ ]:
### Save the output inside Google Drive
drive.mount('drive')

final_data.to_csv('/content/drive/My Drive/data_lake/alpha_raking.csv', encoding='utf-8', index=False)

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [ ]:
final_data

,ticker,qtd_dias,s_p,acwx,emm,cmmdt,usd_real,mkt,size,value,momnt,liq,return,vol,alpha,r_score,last_update,Div. Yield Med
0,ISAE4.SA,4027,0.020,0.030,0.035,0.005,-0.016,0.182,0.022,0.041,0.001,0.014,3.825,0.01835,0.00092,0.090,2025-06-15,8.97
2,SAPR4.SA,4027,0.031,0.032,0.023,-0.012,-0.052,0.242,0.057,0.070,0.010,0.047,3.263,0.02216,0.00069,0.109,2025-06-15,5.23
3,FRAS3.SA,4027,0.031,0.012,0.012,0.013,-0.021,0.179,0.095,-0.007,-0.004,0.071,2.790,0.03954,0.00067,0.021,2025-06-15,3.06
4,RANI3.SA,4027,0.044,0.045,0.035,0.004,-0.033,0.179,0.056,-0.006,-0.018,0.036,2.628,0.03086,0.00060,0.035,2025-06-15,5.13
6,SLCE3.SA,4027,0.029,0.043,0.029,0.049,-0.051,0.183,0.089,0.015,0.018,0.029,2.543,0.02271,0.00056,0.076,2025-06-15,4.44
7,TAEE11.SA,4027,0.011,0.015,0.022,0.001,-0.013,0.153,0.019,0.029,-0.005,0.004,2.380,0.01491,0.00056,0.087,2025-06-15,8.27
10,SHUL4.SA,4027,0.027,0.036,0.020,0.019,-0.049,0.229,0.098,0.051,-0.016,0.052,2.381,0.02343,0.00053,0.096,2025-06-15,4.59
11,PSSA3.SA,4027,0.026,0.039,0.049,-0.016,-0.062,0.230,0.015,0.006,-0.030,-0.003,2.448,0.01830,0.00051,0.143,2025-06-15,4.37
13,BBSE3.SA,3162,0.023,0.045,0.064,-0.010,-0.046,0.306,-0.019,0.004,-0.043,-0.052,1.579,0.01792,0.00044,0.263,2025-06-15,5.87
14,VIVT3.SA,4027,0.028,0.031,0.042,-0.004,-0.021,0.198,-0.007,0.046,-0.023,0.012,2.160,0.01758,0.00044,0.120,2025-06-15,8.49
